# Pràctica 6: extracció d'informació
En aquesta pràctica analitzarem de manera no supervisada un corpus de textos per a extraure informació.\
Primer extraurem i analitzarem les entitats que formen el corpus, a continuació extraurem les paraules claus de cada document i finalment realitzarem un *topic *modeling* del corpus.\
Usem el conjunt de crítiques de cinema de "Mundocine" que estan en format XML dins d'un directori.  
Definim una funció per a extraure el text de la crítica de cada arxiu XML del directori mitjançant una funció de tipus **generator*

In [11]:
import os, re
from xml.dom.minidom import parseString

def parse_folder(path):
    """generator that reads the contents of XML files in a folder
    Returns the <body> of the <review> in each XML file.
    XML files encoded as 'latin-1'"""
    for file in sorted([f for f in os.listdir(path) if f.endswith('.xml')],
                        key=lambda x: int(re.match(r'\d+',x).group())):
        with open(os.path.join(path, file), encoding='latin-1') as f:
            doc=parseString(re.sub(r'(<>)|&|(<-)', '', f.read()))

            titulo = doc.documentElement.attributes["title"].value

            btxt = ""
            review_bod = doc.getElementsByTagName("body")
            if len(review_bod) > 0:
                for node in review_bod[0].childNodes:
                    if node.nodeType == node.TEXT_NODE:
                        btxt += node.data + " "

            rtxt = ""
            review_summ = doc.getElementsByTagName("summary")
            if len(review_summ) > 0:
                for node in review_summ[0].childNodes:
                    if node.nodeType == node.TEXT_NODE:
                        rtxt += node.data + " "
                #separamos después de ciertos signos de puntuación
                rtxt = re.sub(r"([\.\?])", r"\1 ", rtxt)
                        
            rank = int(doc.documentElement.attributes["rank"].value)
            
            yield titulo, rtxt, btxt, rank


## Extracció d'entitats
Analitzarem les entitats (tipus i quantitat) que apareixen en cada document del *corpus*
### Exercici
Construeix una funció de tipus *generator* que retorne en cada iteració les entitats del següent document. Aquestes entitats es generaran com un *string* amb les etiquetes de cada entitat en el text separat per comes, p. ej:
```
'MISC PER MISC MISC MISC MISC ORG PER PER MISC LOC'
```

In [12]:
import spacy

nlp = spacy.load("es_core_news_sm")

def extraer_ner(texto):
    doc = nlp(texto)
    ent_labels = [ent.label_ for ent in doc.ents]
    return ' '.join(ent_labels)

Prova el seu funcionament sobre el text de la primera crítica en el Corpus.

In [13]:
corpus = parse_folder('criticas/train/')
texto = extraer_ner(next(corpus)[2])
texto


'MISC PER PER MISC ORG PER LOC PER PER MISC PER MISC ORG'

Combinarem totes dues funcions per a processar tot el corpus.\
Construeix una funció de tipus *generator* que a partir d'un directori amb les crítiques, processe tots els seus arxius i retorne en cada iteració el llistat d'entitats del següent document

In [14]:
def criticas_ner(folder):
    corpus = parse_folder(folder)
    for critica in corpus:
        yield extraer_ner(critica[2])

Provem el seu funcionament amb el primer arxiu:

In [15]:
criticas_gen = criticas_ner("criticas/train")
next(criticas_gen)

'MISC PER PER MISC ORG PER LOC PER PER MISC PER MISC ORG'

Una vegada s'ha consumit un arxiu, el generador passa el següent i no es pot tornar a l'inici del iterador:

In [16]:
next(criticas_gen)

'MISC PER PER LOC MISC PER PER MISC PER PER LOC PER LOC PER MISC PER MISC LOC LOC MISC PER PER MISC LOC'

## Anàlisi de les entitats
Com a anàlisi molt simple, veurem quantes entitats apareixen en tot el corpus.\
Per a això, comptem el núm. d'entitats de cada document amb el vectorizador BoW de la llibreria `scikit-learn`.
### Exercici
Aplica el vectorizador BoW per a obtindre la seua matriu d'ocurrència. Suma el total de vegades que ha aparegut cada entitat i mostra'l en un dataframe de Colles.

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

vect = CountVectorizer()
bow = vect.fit_transform(criticas_gen)

totales = bow.sum(axis=0)
data = {'Entidad' : vect.get_feature_names_out(), 'Apariciones' : totales.tolist()[0] }

df = pd.DataFrame(data)
df

,Entidad,Apariciones
0,loc,9593
1,misc,17664
2,org,2433
3,per,29604


## Extracció de paraules clau
En aquest exercici extraurem les paraules clau de cada crítica mitjançant la llibreria `textacy`. Després crearem un BoW d'aquests termes per a analitzar amb quins freqüència apareixen.\
Com les paraules claus determinades per la llibreria `textacy` poden ser n-grames, cal unificar els seus constituents amb `_` per a considerar-los termes únics en el vocabulari.  

### Exercici
Defineix una funció per a unir diversos n-*gramas en un únic terme.\
Defineix una funció que extraga les paraules clau d'un document i les retorne com una llista de *tokens*. Utilitza l'algorisme `textrank` sobre el text normalitzat en minúscules sense lematizar.

In [18]:
from textacy.extract import keyterms as kt

def unir_ngramas(texto):
    """Une todos los términos de un n-grama mediante '_'
    para que formen un único término en el vocabulario"""
    
    return texto.replace(' ', '_')

def extraer_keywords(texto, topn=10):
    """Extrar las palabras clave de un texto mediante la
    librería textacy
    Devuelve los topn términos clave como lista de strings"""
    # Y esto no se si esta bien ?????????????
    doc = nlp(texto)
    keywords = kt.textrank(doc=doc, topn=topn)
    return [unir_ngramas(kTerm) for kTerm, rank in keywords]

Provem sobre la crítica descarregada abans...

In [19]:
corpus = parse_folder('criticas/train/')
unir_ngramas(next(corpus)[2])

'Cada_vez_me_gusta_menos_el_cine_de_masas._Las_peliculas_que_ven_todo_el_mundo_me_parecen_cada_vez_mas_coñazo_y_mas_insufribles._No_se_porqué_pero_siempre_el_prota_es_tonto_del_culo_y_tiene_suerte,_y_al_final_de_la_peli,_cuando_ha_logrado_vencer_al_mal,_se_convierte_en_listo,_y_las_chorradas_que_hacia_al_comienzo_de_la_pelicula_se_esfuman_como_por_arte_de_magia._Se_vuelve_maduro_e_inteligente.Esta_peli_de_Spielberg_es_mas_de_lo_mismo,_huir_y_huir_y_que_no_le_den_ni_un_solo_tiro._Además_el_cabron_ha_metido_a_un_par_de_actores_que_es_como_para_echarles_de_comer_aparte._La_niña,_una_vieja_metida_en_el_cuerpo_de_una_niña,_porque_solo_hay_que_verle_hablar_(en_version_original_claro)_para_darse_cuenta_que_estamos_ante_uno_de_los_grandes_freaks_del_cine._Se_creeran_que_hace_gracia_la_nena_cuando_habla_igual_que_su_puta_madre,_pero_a_mi_me_causa_pavor._Ver_a_una_cria_que_habla_como_una_persona_madura_es_algo_horroroso._Los_niños_son_niños_y_verlos_fuera_de_su_rol_asusta.Luego_esta_el_hijo_adol

### Exercici
Defineix una funció *generator* que retorne iterativament per a cada crítica dins d'un directori les seues paraules clau.
Ha de retornar per cada document un *string* amb les paraules clau separades per espai, per a poder usar-ho amb el vectorizador de `scikit-learn`

In [20]:
def criticas_keyword(folder):
    #COMPLETAR
    corpus = parse_folder(folder)
    for critica in corpus:
        yield ' '.join(extraer_keywords(critica[2]))
    

Provem sobre el primer document del corpus...

In [21]:
#COMPLETAR
gen = criticas_keyword('criticas/train/')
next(gen)

'puta_peli puta_madre viejo_metida persona_maduro version_original pelicula_aparezca manipulacion_militar cine ningun_arma efecto_especial'

### Anàlisi de les paraules clau
Implementa un vectorizador BoW per a comptar les paraules clau que apareixen almenys en un 0,5% dels documents i mostra-les ordenades en ordre descendent de freqüència (paraula, núm. d'aparicions en total) dins d'un *DataFrame. Mostra només els 10 termes més utilitzats.

In [22]:
#COMPLETAR
vec = CountVectorizer(min_df=0.005)

gen = criticas_keyword('criticas/train/')

tokenized_corpus = vec.fit_transform(gen)
tokenized_corpus = tokenized_corpus.toarray()
df = tokenized_corpus.sum(axis=0) #Se pone toarray porque tenemos una matriz sparse, y nos interesa una normal para sumar las columnas (axis=0)
vocab = vec.get_feature_names_out()

dataFrame = pd.DataFrame({'término':vocab, 'freq':df})
sortedDataFrame = dataFrame.sort_values(by='freq', ascending=False)

top_10 = sortedDataFrame.head(10)
top_10

,término,freq
35,película,347
9,cine_español,86
26,ldquo,68
14,efecto_especial,60
30,mejor_película,51
33,obra_maestro,51
3,banda_sonoro,43
43,ser_humano,41
5,cine,40
42,ser,40


### Exercici
També ho podem fer amb la llibreria `gensim`, per a així poder comptar el núm. de documents en el qual apareix cada keyword (amb l'atribut `dfs` de l'objecte `Dictionary`).\
Defineix de nou la funció `critiques_keyword` perquè retorne les paraules clau de cada document en un format compatible amb Gensim i calcula el seu diccionari BoW (no fa falta calcular la matriu de tots els documents).\
Després, utilitza el mètode `filter_extremes` per a quedar-te amb els termes que apareixen almenys en un 0,5% dels documents (nota: usa l'atribut `.num_docs` per a calcular el núm. de documents que formen el 0,5% del total)\
Finalment, usa l'atribut `dfs` del diccionari per a mostrar les paraules clau i el núm. de documents en el qual s'usen, com *dataframe. Mostra només els 10 termes més freqüents.

In [23]:
from gensim.corpora import Dictionary

#COMPLETAR
def criticas_keyword(folder):
    #COMPLETAR
    array_tokens = []
    corpus = parse_folder(folder)
    for i in range(1,20): #Para que no los recorra todos
        tokens = nlp(next(corpus)[2])
        filtered_tokens = [t.lower_ for t in tokens if
                       len(t.text)>3 and
                       not t.is_punct] #Limpiamos un poco el texto
        array_tokens.append(list(filtered_tokens))

    return array_tokens #devolvemos los tokens como generaciones.
        
genTokens = criticas_keyword('criticas/train/')
dic = Dictionary(genTokens)
num_doc_min = dic.num_docs * 0.005
dic.filter_extremes(no_below = num_doc_min)
mapped_corpus = list(map(dic.doc2bow, tokenized_corpus))

#Contamos los terminos y la frecuencia
terminos = []
freq = []
for i in dic.dfs:
    terminos.append(dic[i])
    freq.append(dic.dfs[i])
  
#Generamos el dataframe y lo ordenamos, viendo los 10 primeros  
dataFrame = pd.DataFrame({'término':terminos, 'freq':freq})
sortedDataFrame = dataFrame.sort_values(by='freq', ascending=False)

top_10 = sortedDataFrame.head(10)
top_10

,término,freq
245,personajes,9
253,vida,9
139,nada,9
17,cuando,9
33,mismo,9
249,poco,9
434,está,9
73,algo,9
48,porque,9
37,además,9


## Topic modeling
Finalment, calcularem les temàtiques del corpus de crítiques mitjançant un model de *topic modeling* usant l'algorisme LDA.

In [24]:
# Gensim
import gensim
from gensim.models import LdaModel
from pprint import pprint
import warnings


# herramientas de dibujado
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

import matplotlib.pyplot as plt
%matplotlib inline

In [25]:
warnings.filterwarnings("ignore", category=DeprecationWarning)

Definim una classe *iterable* per a obtindre els documents del Corpus línia a línia des de l'arxiu del conjunt d'exemple i convertir-los en un llistat de tokens. Al contrari que els generadors, les classes *iterables* poden tornar a l'inici de la llista cada vegada, i no s'esgoten quan es consumeixen tots els elements.
### Exercici
Per a calcular les temàtiques, utilitzarem els lemes de cada terme del corpus, considerant només aquells la funció morfològica dels quals siga nom, nom propi, adjectiu, verb o adverbi. Defineix una funció `lematize_doc` que retorne per a cada document un llistat dels tokens amb aquest processament.

In [26]:
def lemmatize_doc(text, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV', 'PROPN']):
    """Función que devuelve la lista de lemas en minúscula de una string,,
    excluyendo las palabras cuyo POS_TAG no está en la lista allowed_postags.
    Considera sólo lemas de más de 3 caracteres y omite las stop words."""
    #COMPLETAR
    doc = nlp(text.lower())
    lemmas = []
    for token in doc:
        if token.pos_ in allowed_postags and token.is_stop == False and len(token.text) > 3:
            lemmas.append(token.lemma_)
    return lemmas    

def criticas_tokens(folder):
    for c in parse_folder(folder):
        yield lemmatize_doc(c[2])
            
#para no tener que cargar todo el corpus en memoria creamos un streamer
class BOW_Corpus(object):
    """
    Iterable: en cada iteración devuelve el vector bag-of-words
    del siguiente documento en el corpus.
    El corpus es el listado de críticas alojadas en el directorio
    pasado como argumento al instanciar la clase.
    
    Procesa un documento cada vez, así
    nunca carga el corpus entero en RAM.
    """
    def __init__(self, dirname):
        self.dirname = dirname
        #crea el diccionario = mapeo de documentos a sparse vectors
        self.diccionario = gensim.corpora.Dictionary(criticas_tokens(self.dirname))
        
    def __len__(self):
        #necesitamos saber la longitud del corpus para visualizar con pyLDAvis
        return self.diccionario.num_docs
    
    def __iter__(self):
        """
        __iter__ es un iterable => BOW_Corpus es un streamed iterable.
        """
        for tokens in criticas_tokens(self.dirname):
            # transforma cada doc (lista de tokens) en un vector sparse uno a uno
            yield self.diccionario.doc2bow(tokens)

Crea la matriu BoW sobre el corpus com un element de la classe BOW_Corpus amb el nom `bow_critiques`

In [27]:
#COMPLETAR
bow_criticas = BOW_Corpus(dirname='criticas/test/')

Mostrem el BoW del primer document com a comprovació

In [28]:
for b in bow_criticas:
    print(b)
    break

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 2), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 2), (22, 1), (23, 2), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 2), (32, 1), (33, 2), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 5), (42, 1), (43, 1), (44, 2), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 2), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 2), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 2), (87, 1), (88, 7), (89, 1), (90, 1), (91, 1), (92, 3), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 1), (107, 1), (108, 1), (109, 5), (110, 1),

In [29]:
#términos en el diccionario
len(bow_criticas.diccionario.token2id)

33212

In [30]:
#longitud del corpus
len(bow_criticas)

1164

### Exercici
Crea un model LDA sobre el corpus de crítiques amb 5 temes i mostra'l gràficament usant les funcions de la llibreria `pyLDAvis`

In [31]:
#COMPLETAR
lda = LdaModel(corpus=bow_criticas, num_topics=5)

# Crear una visualización interactiva del modelo LDA
vis = gensimvis.prepare(lda, bow_criticas, bow_criticas.diccionario)

# Mostrar la visualización en el navegador
pyLDAvis.display(vis)